# First, import the py11 module

The underlying technology that makes this work on pybind11 (https://github.com/pybind/pybind11)

It compiles individual functions in C++ to provide an more dynamic approach to C++ programming.

First step, import the following symbols...

In [ ]:
!rm -fr ~/tmp
!mkdir -p ~/tmp

In [ ]:
from py11 import py11, smap, svec

To create a C++ function that's callable from Python, use the @py11 decorator and
put your code in the docstring. You
can supply it with a list of headers needed. You can modify the source and
re-execute the cell if you want different output.

In [ ]:
@py11(headers=['<iostream>'])
def hello():
    """
    std::cout << "Hello, world" << std::endl;
    std::cerr << "Goodbye, world" << std::endl;
    """

We can now call this function directly from Python. It won't recompile again unless we change the source code.

In [ ]:
hello()

We can pass arguments to the function and receive arguments back. To do this, we use
Python's syntax for declaring types.

In [ ]:
@py11()
def sumi(a : int,b : int)->int:
    """
    // add 2 args
    return a+b;
    """

In [ ]:
sumi(20,22)

Static variables can be used to remember state.

In [ ]:
@py11()
def count()->int:
    """
    static int counter = 0; // remember state
    return counter++;
    """

In [ ]:
for i in range(5):
    print(count())

If you want to pass a more complex data structure, like an std::vector, use svec.

In [ ]:
@py11(headers=["<vector>"])
def sumv(v : svec[int])->int:
    """
    int sum=0;
    for(auto i=v.begin(); i != v.end(); ++i)
        sum += *i;
    return sum;
    """

In [ ]:
sumv([1,2,3,4])

Something similar works for maps...

In [ ]:
@py11(header=['<map>'])
def summ(m : smap[str,int])->int:
    """
    int sum = 0;
    for(auto i=m.begin();i != m.end();++i) {
        sum += i->second;
    }
    return sum;
    """

print(summ({"a":3,"b":10}))

It is possible to use pybind11 to throw an exception from C++ into Python.

In [ ]:
@py11()
def get_and_set(k : str, v : int, p : bool)->int:
    """
    static std::map<std::string,int> dict;
    if(!p)
        if(dict.find(k) == dict.end())
            throw py::key_error(k);
    int r = dict[k];
    if(p)dict[k] = v;
    return r;
        
    """

In [ ]:
get_and_set("a",3,True)

In [ ]:
get_and_set("a",0,False)

In [ ]:
try:
    get_and_set("b",0,False)
except KeyError as ke:
    print("Not a valid key:",ke)

Recursion works...

In [ ]:
@py11()
def fib(n : int)->int:
    """
    if(n < 2) return n;
    return fib(n-1)+fib(n-2);
    """

If we define a Python version of this same function, we can run a benchmark...

In [ ]:
def fib2(n):
    if n < 2:
        return n
    return fib2(n-1)+fib2(n-2)

In [ ]:
def timer(fun,*args):
    from time import time
    t1 = time()
    fun(*args)
    t2 = time()
    print("time:",t2-t1)

You should find that fib is much faster than fib2.

In [ ]:
timer(fib,34)
timer(fib2,34)

If we want to call one @py11() function from another, we can. However, we need to specify what
we are doing by means of the funs parameter.

In [ ]:
@py11(funs=[fib],headers=["<iostream>"])
def print_fib(n:int)->None:
    """
    std::cout << "fib(" << n << ") = " << fib(n) << std::endl;
    """

In [ ]:
print_fib(15)

Note that if we redefine fib, print_fib() will automatically
update to use the new version.

In [ ]:
@py11()
def fib(n : int)->int:
    """
    if(n < 0) return n;
    return fib(n-1)+fib(n-2);
    """

In [ ]:
print_fib(15)

If we want to change the compile flags, we can.

In [ ]:
from py11 import py11, create_type, set_flags

# Set the compile flags
set_flags("-std=c++17 -L/usr/local/lib64 -lhpx -Wl,-rpath=/usr/local/lib64")

# Create your own types
create_type("func","std::function<void()>")

Sometimes you need to set the LD_LIBRARY_PATH

HPX is an advanced parallel threading library. However, to use it, you have to have a
special threading environment. To make this work, we will create a "wrapper function."
This function needs to take a std::function<void()> as an input argument.

In [ ]:
@py11(headers=["<run_hpx.cpp>"],recompile=True)
def hpx_wrapper(f : func)->None:
    """
    const char *num = getenv("HPX_NUM_THREADS");
    int num_threads = num == 0 ? 4 : atoi(num);
    std::cout << "Using " << num_threads << " threads." << std::endl;
    hpx_global::submit_work(num_threads,f);
    """

In [ ]:
@py11(headers=["<hpx/hpx.hpp>"],recompile=True,wrap=hpx_wrapper)
def do_fut()->None:
    """
    auto f = hpx::async([](){ return 5; });
    std::cout << "f=" << f.get() << std::endl;
    """

In [ ]:
do_fut()

In [ ]:
create_type("future","hpx::future",is_template=True)

In [ ]:
@py11(headers=["<hpx/hpx.hpp>"],recompile=True,wrap=hpx_wrapper)
def hpx_fib(n : int)->int:
    """
    if(n < 2)
        return n;
    if(n < 25)
        return hpx_fib(n-1)+hpx_fib(n-2);
    hpx::future<int> f1 = hpx::async(hpx_fib,n-1);
    int f2 = hpx_fib(n-2);
    return f1.get() + f2;
    """

In [ ]:
nfib = 34
(hpx_fib(nfib), fib2(nfib))

In [ ]:
nfib = 34
timer(hpx_fib,nfib)
timer(fib2,    nfib)

In [ ]:
def timer2(fun,args,zargs):
    from time import time
    t1 = time()
    fun(*args)
    t2 = time()
    fun(*zargs)
    t3 = time()
    
    del1 = t2-t1 # time with args
    del2 = t3-t2 # time with zargs
    
    delt = del1 - del2
    print("time:",delt)

In [ ]:
nfib = 34
timer2(hpx_fib,[nfib],[1])
timer2(fib2,   [nfib],[1])

Sometimes calling C++ can kill the notebook kernel. To avoid that problem, we can run in a thread.

In [ ]:
def run_fork(f,*args):
    import os
    pid = os.fork()
    if pid==0:
        f(*args)
        os._exit(0)
    else:
        while True:
            wpid, wstatus = os.wait()
            if wpid == pid:
                if wstatus == 0:
                    pass
                print("status:",wstatus)
                return

In [ ]:
@py11()
def segv():
    """
    int *i=0;
    i[0]=1;
    """

In [ ]:
run_fork(segv)

In [ ]:
@py11(headers=["<iostream>"])
def prvec(v : svec[float])->None:
    """
    std::cout << "prvec" << std::endl;
    for(auto i : v) 
      std::cout << i << " ";
    std::cout << std::endl;
    """
prvec([1.0,2.0,3])

Now for some fancy stuff... defining executors that (in principle) can work on more than one locality

In [ ]:
create_type("executor","hpx::compute::host::block_executor<>")

@py11(headers=["<hpx/hpx.hpp>","<hpx/include/compute.hpp>"])
def getexec()->executor:
    """
    auto host_targets = hpx::compute::host::get_local_targets();
    typedef hpx::compute::host::block_executor<> executor_type;
    executor_type exec(host_targets);
    
    for(auto host : host_targets)
        std::cout << host.get_locality() << std::endl;
        
    return exec;
    """

Fill an array

In [ ]:
@py11(wrap=hpx_wrapper,funs=[prvec,getexec],headers=[
    "<hpx/hpx.hpp>",
    "<hpx/include/parallel_fill.hpp>",
    "<hpx/include/compute.hpp>",
    "<hpx/include/parallel_executors.hpp>"
    ])
def fill_example()->None:
    """
    auto exec = getexec();
    
    std::vector<float> vd;
    for(int i=0;i<5;i++) vd.push_back(1.f);
    prvec(vd);
    hpx::fill(hpx::execution::par.on(exec),vd.begin(),vd.end(),2.0f);
    prvec(vd);
    """

fill_example()

In [ ]:
@py11(wrap=hpx_wrapper,funs=[prvec,getexec],headers=[
    "<hpx/hpx.hpp>",
    "<hpx/parallel/algorithms/reverse.hpp>",
    "<hpx/include/compute.hpp>",
    "<hpx/include/parallel_executors.hpp>"
    ])
def rev_example()->None:
    """
    auto exec = getexec();
    
    std::vector<float> vd;
    for(int i=0;i<10;i++) vd.push_back(i);
    prvec(vd);
    hpx::reverse(hpx::execution::par.on(exec),vd.begin(),vd.end());
    prvec(vd);
    """

rev_example()

In [ ]:
@py11(wrap=hpx_wrapper,funs=[prvec,getexec],headers=[
    "<hpx/hpx.hpp>",
    "<hpx/include/parallel_minmax.hpp>",
    "<hpx/include/compute.hpp>",
    "<hpx/include/parallel_executors.hpp>"
    ])
def min_example()->None:
    """
    auto exec = getexec();
    
    std::vector<float> vd;
    for(int i=0;i<10;i++) vd.push_back(i+300);
    prvec(vd);
    auto ptr = hpx::min_element(hpx::execution::par.on(exec),vd.begin(),vd.end(),std::less<float>());
    std::cout << *ptr << std::endl;
    ptr = hpx::max_element(hpx::execution::par.on(exec),vd,std::less<float>());
    std::cout << *ptr << std::endl;
    """

min_example()

Exercise: Make this routine run in parallel with HPX

In [ ]:
@py11(wrap=hpx_wrapper,headers=["<hpx/hpx.hpp>"])
def myqsort(v : svec[int])->svec[int]:
    """
    if(v.size() < 2)
        return v;
    int pivot = v[rand() % v.size()];
    std::vector<int> lo, hi, eq;
    for(int i=0;i<v.size();i++) {
        if(v[i] < pivot)
           lo.push_back(v[i]);
        else if(v[i] > pivot)
           hi.push_back(v[i]);
        else
           eq.push_back(v[i]);
    }
    lo = myqsort(lo);
    hi = myqsort(hi);
    std::vector<int> result;
    result.insert(result.end(),lo.begin(),lo.end());
    result.insert(result.end(),eq.begin(),eq.end());
    result.insert(result.end(),hi.begin(),hi.end());
    return result;
    """

from random import randint
inp = [randint(1,100) for v in range(20)]
out = myqsort(inp)
print(inp)
print(out)

# check result...
for i in range(1,len(out)):
    assert out[i-1] <= out[i]
vals = {}
for v in inp:
    if v not in vals:
        vals[v] = 0
    vals[v] += 1
for v in out:
    # if vals[v] is not defined, our sort messed up
    vals[v] -= 1
for v in vals.values():
    assert v == 0 # should have all the same values in output